# Introduction to the `Mps` and `Mpo` structures and operations between them

In [1]:
using MPStates, LinearAlgebra, Test

# `Mps` structure

An `Mps` is a structure that is composed of three fields:
1. `L` : an integer representing the number of sites of the MPS.
2. `d` : an integer representing the physical dimension of the MPS.
3. `M` : a vector of rank-3 tensors that contains the information of the MPS.

The indices of the `M` tensors are: `M[i1, i2, i3]`:
1. `i1`: bond dimension connecting `M[i]` with `M[i-1]`.
2. `i2`: physical dimension.
3. `i3`: bond dimension connecting `M[i]` with `M[i+1]`.


In Julia you can access the fields of a structure as, e.g.: `psi.L`.

Let's create our first `Mps`. We must choose the bond dimension, the type of the elements of the `Mps` (`Float64` or `ComplexF64`) and the physical bond dimension. Additionally we can either start a random `Mps` or start an special MPS from the already defined ones: `"W"`, the W state; `"GHZ"`, the GHZ state; `"full"`, the state: $|0\cdots 00> + |0\cdots 01> + |0\cdots 10> + \cdots + |1\cdots 10> + |1\cdots 11>$ ; or `"product"`, the product state $|00\cdots 00>$. All of these predefined states have physical dimension `d=2`.

In [2]:
L = 10
# Type of the Mps.
T = Float64
# Physical dimension.
d = 2

psi = init_mps(Float64, L, "random", d);

## Expectation values of `Mps`
We can already start to measure some observables of the `Mps` using the `expected` function (no need to remember more names thanks to excellent Julia's multiple dispatch!)

In [3]:
# Compute the occupation number at site 5. We can either input the observable (operator) as
# a string or as a full matrix. 
ex1 = expected(psi, "n", 5)
ex2 = expected(psi, [[0. 0.]; [0. 1.]], 5)
@test ex1 ≈ ex2

# Correlations can be easily measured too!
ex3 = expected(psi, "b+", 1, "b", 3)
ex4 = expected(psi, [[0. 1.]; [0. 0.]], 1, [[0. 0.]; [1. 0.]], 3)
@test ex3 ≈ ex4

# If the operators are fermionic creation and annihilation operators we have to indicate it
# in the `ferm_op` argument, which is just the fermionic parity operator `I - 2n`.
ex5 = expected(psi, "c+", 1, "c", 3, ferm_op="Z")
ex6 = expected(psi, [[0. 1.]; [0. 0.]], 1, [[0. 0.]; [1. 0.]], 3, ferm_op=[[1. 0.]; [0. -1.]])
@test ex5 ≈ ex6

# When two operators act on the same site, `expected` multiplies them and makes them act on
# that site.
ex7 = expected(psi, "c+", 3, "c", 3)
ex8 = expected(psi, "n", 3)
@test ex7 ≈ ex8

Test Passed

Implemented string operators are for `d=2`: `"n"`, occupation operator; `"Z"`, fermionic parity operator, `I(2) - 2n`; `"a+"`, `"b+"`, `"c+"`, particle creation operators, they all share the same matrix representation, the distinction between fermions or hard-core bosons is made through the `ferm_op` argument; `"a"`, `"b"`, `"c"`, particle annihilation operators. For `d=3` (warning: not enough tested yet): `"Sz"`, `"S+"`, and `"S-"`.

# `Mpo` structure 

Once we have defined the `Mps`, we want to build the Hamiltonian. This will be an `Mpo` structure, whose fields are:
1. `L` : an integer representing the number of sites of the MPO.
2. `d` : an integer representing the physical dimension of the MPO.
3. `W` : a vector of rank-4 tensors that contains the information of the MPO.

The indices of the `W` tensors are: `W[i1, i2, i3, i4]`:
1. `i1`: bond dimension connecting `W[i]` with `W[i-1]`.
2. `i2`: physical dimension connecting `W[i]` with `psi[i]`.
3. `i3`: physical dimension connecting `W[i]` with `conj(psi[i])`.
4. `i4`: bond dimension connecting `W[i]` with `W[i+1]`.

## Building an `Mpo`

We start by creating a `0` operator with `init_mpo(T, L, d)`, with `T` the type of the elements of the MPO, `L` the number of sites, and `d` the physical dimensions.

Once we have created an initial MPO we continue by adding operators to it. This can be done in a very general way using the `add_ops!` function, with a notation similar to `expected`. Let's build a Hubbard Hamiltonian like

$$ H = 0.5\sum^{L-1}_{i=1} c^\dagger_i c_{i+1} + 0.7\sum^{L-1}_{i=1} n_i n_{i+1} $$

In [4]:
# Let's first define the matrices that describe the Hamiltonian.
# Hopping matrix.
J = 0.5*Symmetric(diagm(1 => ones(L-1)))
# Interaction matrix.
V = 0.7*diagm(1 => ones(L-1))

# Build the Hamiltonian.
H = init_mpo(T, L, d)
add_ops!(H, "c+", "c", J, ferm_op="Z")
add_ops!(H, "n", "n", V)
# Similar to `expected`, we could have writen: add_ops!(H, [[0. 0.]; [0. 1.]], [[0. 0.]; [0. 1.]], V);

This way of building the MPO with `add_ops!` is still a bit inefficient in which it creates a big array of tensors with many zero elements, which will slow down the DMRG algorithms. This will be fixed soon. However, by now you can use `H = init_mpo(T, J, V, is_fermionic=is_fermionic)` to create an MPO that is more efficient in the storage of the elements, but it is only valid for `d=2` and for interactions and hoppings as described above. This function will be deprecated when a more efficient version of `add_ops` is available.

# Expectation values of an `Mpo`

This can also be done with the `expected` function, but now the notation changes a bit because it is more natural to write the `Mpo` first. This may change in future versions.

In [5]:
exp = expected(H, psi)

3.2893246642952336